In [26]:

from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.document_loaders import GitLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from scrape_and_load_langchain import PythonClassDirectoryLoader
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent, Tool, StructuredChatAgent, AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain.agents.self_ask_with_search.base import SelfAskWithSearchAgent




PINECONE='5d4ffa0d-2615-4348-ac65-a3071404be6b'

def file_filter(file_path: str) -> bool:
    return True if file_path.endswith(".md") else False

MessagesPlaceholder(variable_name='chat_history')

In [3]:
embeddings = OpenAIEmbeddings(allowed_special="all")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
class_loader = PythonClassDirectoryLoader("/Users/josephblazick/Documents/langchain", glob="**/*.py")
class_loader.load()
git_loader = GitLoader(repo_path="/Users/josephblazick/Documents/langchain", branch="v0.0.196", file_filter=file_filter)
package_docs = git_loader.load_and_split()

In [4]:
docsearch = FAISS.from_documents(package_docs, embeddings)

In [17]:
def qa_docs(question: str):
    """
    Answers your question using the markdown files found in the langchain documeentation repository.
    This function is best used when you want examples of how to use particular classes or functions.
    """
    documents = docsearch.similarity_search(question, k = 10)
    qa_chain = load_qa_chain(ChatOpenAI(temperature=0, model="gpt-3.5-turbo"), chain_type="refine")
    return qa_chain.run({"input_documents": documents, "question": question})

def summary_docs(question: str):
    """
    Give a summary of documents retrieved from the markdown files found in the langchain documeentation repository.
    This function is best used when you want a general summary of how to use langchain modules or classes.
    """
    documents = docsearch.similarity_search(question, k = 10)
    sum_chain = load_summarize_chain(ChatOpenAI(temperature=0, model="gpt-3.5-turbo"), chain_type="map_reduce")
    return sum_chain.run({"input_documents": documents, "question": question})

def class_lookup(item: str):
    """
    Lookup for class definitions in the langchain codebase. You should use this function when you need to see the source code
    related to a particular class.
    """
    return class_loader.get_class(item)


tools = [
    Tool.from_function(
        func=qa_docs,
        name="Questions about Langchain codebase",
        description="Answers your question using the markdown files found in the langchain documeentation repository. This function is best used when you want examples of how to use particular classes or functions. Your input should be as descriptive as possible to ensure documents returned are as relevant as possible.",
    ),
    Tool.from_function(
        func=summary_docs,
        name="Summarise langchain docs",
        description="Give a summary of documents retrieved from the markdown files found in the langchain documeentation repository. This function is best used when you want a general summary of how to use langchain modules or classes. Your input should be as descriptive as possible to ensure documents returned are as relevant as possible.",
    ),
    Tool.from_function(
        func=class_lookup,
        name="LangChain Class Lookup",
        description="Lookup for class definitions in the langchain codebase. You should use this function when you need to see the source code related to a particular class",
    )
]


In [ ]:

prompt = SelfAskWithSearchAgent.create_prompt()

In [34]:

agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.SELF_ASK_WITH_SEARCH, 
    llm=ChatOpenAI(temperature=0.5, model="gpt-4"), 
    verbose=True,
    memory=memory,
)

# planner = load_chat_planner(ChatOpenAI(temperature=0.5, model="gpt-4"))
# executor = load_agent_executor(ChatOpenAI(temperature=0.0, model="gpt-4"), tools=tools, verbose=True)
# agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [35]:
agent.run(
    '''
    How do i access the prompt of this? The initialize_agent function returns a AgentExecutor object.
    from langchain.agents import AgentType, initialize_agent, Tool

    agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.SELF_ASK_WITH_SEARCH, 
    llm=ChatOpenAI(temperature=0.5, model="gpt-4"), 
    verbose=True,
)
    '''
)



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `I should find an example of how to access the prompt using the AgentExecutor object in the langchain documentation.`

In [44]:
# print(agent.agent.llm_chain.prompt.template)
executor.run("AttributeError: 'AgentExecutor' object has no attribute 'prompt'")



> Entering new AgentExecutor chain...
I need to find information about the 'AgentExecutor' class and its attributes.
Action: LangChain Class Lookup
Action Input: AgentExecutor
Observation: class AgentExecutor(Chain):
    """Consists of an agent using tools."""
    agent: Union[BaseSingleActionAgent, BaseMultiActionAgent]
    tools: Sequence[BaseTool]
    return_intermediate_steps: bool = False
    max_iterations: Optional[int] = 15
    max_execution_time: Optional[float] = None
    early_stopping_method: str = 'force'
    handle_parsing_errors: Union[bool, str, Callable[[OutputParserException], str]] = False

    @classmethod
    def from_agent_and_tools(cls, agent: Union[BaseSingleActionAgent, BaseMultiActionAgent], tools: Sequence[BaseTool], callback_manager: Optional[BaseCallbackManager]=None, **kwargs: Any) -> AgentExecutor:
        """Create from agent and tools."""
        return cls(agent=agent, tools=tools, callback_manager=callback_manager, **kwargs)

    @root_validator()
 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 62e35a8ac196cebb1fa877b6693ec78f in your message.).


I now know the final answer
Final Answer: The 'AgentExecutor' class does not have a 'prompt' attribute.

> Finished chain.


"The 'AgentExecutor' class does not have a 'prompt' attribute."

In [52]:
print(agent.agent.llm_chain.prompt.template)

AttributeError: 'PlanAndExecute' object has no attribute 'agent'

In [65]:
print(agent.executor.chain.agent.llm_chain.prompt.messages[0].prompt.template)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Questions about Langchain codebase: Answers your question using the markdown files found in the langchain documeentation repository. This function is best used when you want examples of how to use particular classes or functions., args: {{{{'tool_input': {{{{'type': 'string'}}}}}}}}
Summarise langchain docs: Give a summary of documents retrieved from the markdown files found in the langchain documeentation repository. This function is best used when you want a general summary of how to use langchain modules or classes, args: {{{{'tool_input': {{{{'type': 'string'}}}}}}}}
LangChain Class Lookup: Lookup for class definitions in the langchain codebase. You should use this function when you need to see the source code related to a particular class, args: {{{{'tool_input': {{{{'type': 'string'}}}}}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_i